In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%env OMP_NUM_THREADS=16

In [ ]:
import random

import dotenv
import nltk
import torch
import seaborn as sns
from tqdm import tqdm

from llm_ol.llm.embed import load_embedding_model, embed

dotenv.load_dotenv()

In [ ]:
model, tokenizer = load_embedding_model()

In [ ]:
wordnet = nltk.corpus.wordnet

synonyms = []

for synset in wordnet.all_synsets():
    if len(synset.lemma_names()) > 1:
        synonyms.append(synset.lemma_names())

In [ ]:
sims = []

for syns in tqdm(random.sample(synonyms, 500)):
    e = embed(syns, model, tokenizer)
    e = e / e.norm(dim=1, keepdim=True)
    sim = e @ e.T
    # remove diagonal
    sim = sim[~torch.eye(sim.shape[0], dtype=torch.bool)].view(-1)
    sims.append(sim)

sims = torch.cat(sims)
sims.shape

In [ ]:
sns.ecdfplot(x=sims.numpy().flatten())